In [1]:
from PIL import Image, ImageEnhance, ImageFilter  
import os
from glob import glob

import pandas as pd
import numpy as np
import random

In [2]:
def create_folder(path, folder = ""):
    
    final_path = os.path.join(path, folder)
    
    if not os.path.isdir(final_path):
        os.makedirs (final_path)
    
    return final_path

In [3]:
def images_count(path_folder):
    lst = os.listdir(path_folder) 
    count = len(lst)
    return count

In [4]:
# https://www.codementor.io/@isaib.cicourel/image-manipulation-in-python-du1089j1u

# Return color value depending on quadrant and saturation
def get_saturation(value, quadrant):
  if value > 223:
    return 255
  elif value > 159:
    if quadrant != 1:
      return 255

    return 0
  elif value > 95:
    if quadrant == 0 or quadrant == 3:
      return 255

    return 0
  elif value > 32:
    if quadrant == 1:
      return 255

    return 0
  else:
    return 0

# Get the pixel from the given image
def get_pixel(image, i, j):
  # Inside image bounds?
  width, height = image.size
  if i > width or j > height:
    return None

  # Get Pixel
  pixel = image.getpixel((i, j))
  return pixel

# Create a dithered version of the image
def convert_dithering(image):
    # Get size
    width, height = image.size

    # Create new Image and a Pixel Map
    new = create_image_white(width, height)
    pixels = new.load()

    # Transform to half tones
    for i in range(0, width, 2):
        for j in range(0, height, 2):
          # Get Pixels
          p1 = get_pixel(image, i, j)
          p2 = get_pixel(image, i, j + 1)
          p3 = get_pixel(image, i + 1, j)
          p4 = get_pixel(image, i + 1, j + 1)

          # Color Saturation by RGB channel
          red   = (p1[0] + p2[0] + p3[0] + p4[0]) / 4
          green = (p1[1] + p2[1] + p3[1] + p4[1]) / 4
          blue  = (p1[2] + p2[2] + p3[2] + p4[2]) / 4

          # Results by channel
          r = [0, 0, 0, 0]
          g = [0, 0, 0, 0]
          b = [0, 0, 0, 0]

          # Get Quadrant Color
          for x in range(0, 4):
            r[x] = get_saturation(red, x)
            g[x] = get_saturation(green, x)
            b[x] = get_saturation(blue, x)

          # Set Dithered Colors
          pixels[i, j]         = (r[0], g[0], b[0])
          pixels[i, j + 1]     = (r[1], g[1], b[1])
          pixels[i + 1, j]     = (r[2], g[2], b[2])
          pixels[i + 1, j + 1] = (r[3], g[3], b[3])

    # Return new image
    return new

In [5]:
def convert_brightness(image, brightness):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness)
    

In [6]:
def image_name_brightness(name, brightness):
    return ''.join([name, "_", f"{brightness}".replace('.', '_')])

In [7]:
def image_name_dithering(name):
    return ''.join([name, "_dither"])

In [8]:
def image_name_low_quality(name):
    return ''.join([name, "_low"])

In [9]:
def save_image(img, path, name, fillcolor,  quality=75, extension='png'):

    
    img = img.rotate(random.randint(5, 175), Image.NEAREST, expand = False, fillcolor = fillcolor)
    img.save(f"{path}/{name}.{extension}", quality=quality)

In [10]:
# Create a new image with the given size
def create_image_white(i, j):
  image = Image.new("RGB", (i, j), "white")
  return image

In [11]:
def create_image(path, name, r, g, b, override = False, width = 128, height = 128, brightness = 1, extension='png'):
    print(path, name, r, g, b, brightness)
    create_folder(path)
    
    name_brightness = image_name_brightness(name, brightness)
    name_dithering = image_name_dithering(name_brightness)
    name_low_quality = image_name_low_quality(name_brightness)
    
    if glob(f"{path}/{image_name_brightness}.{extension}"):
        #print("already exists")
        if override:
            pass
            
    else:
        #print("let's go !")
        img  = Image.new( mode = "RGB", size = (width, height), color = (r, g, b) )
        
        img = convert_brightness(img, brightness)
        save_image(img, path, name_brightness, (r, g, b))
        
         
        save_image(img, path, name_low_quality, (r, g, b), 1)
        
        # Baisse les perfs
        #img = convert_dithering(img)
        #save_image(img, path, name_dithering, (r, g, b))
        
        
    

In [12]:

df = pd.read_csv("./dataset/html_color_names.csv", sep=";")

df.head()

,color,enable,color_name,hex_code,r,g,b,remark
0,red,1,IndianRed,#CD5C5C,205,92,92,NaN
1,red,1,LightCoral,#F08080,240,128,128,NaN
2,red,1,Salmon,#FA8072,250,128,114,NaN
3,red,0,DarkSalmon,#E9967A,233,150,122,NaN
4,red,0,LightSalmon,#FFA07A,255,160,122,NaN


In [13]:
images_folder = "./dataset/images/"

In [14]:

for index, row in df.iterrows():
    for i in range(9, 14):
        if row['enable'] == 1:
            create_image(
                f"{images_folder}{row['color']}", 
                row['color_name'], 
                r = row['r'], 
                g = row['g'], 
                b = row['b'], 
                brightness=i/10)
    

./dataset/images/red IndianRed 205 92 92 0.9
./dataset/images/red IndianRed 205 92 92 1.0
./dataset/images/red IndianRed 205 92 92 1.1
./dataset/images/red IndianRed 205 92 92 1.2
./dataset/images/red IndianRed 205 92 92 1.3
./dataset/images/red LightCoral 240 128 128 0.9
./dataset/images/red LightCoral 240 128 128 1.0
./dataset/images/red LightCoral 240 128 128 1.1
./dataset/images/red LightCoral 240 128 128 1.2
./dataset/images/red LightCoral 240 128 128 1.3
./dataset/images/red Salmon 250 128 114 0.9
./dataset/images/red Salmon 250 128 114 1.0
./dataset/images/red Salmon 250 128 114 1.1
./dataset/images/red Salmon 250 128 114 1.2
./dataset/images/red Salmon 250 128 114 1.3
./dataset/images/red Crimson 220 20 60 0.9
./dataset/images/red Crimson 220 20 60 1.0
./dataset/images/red Crimson 220 20 60 1.1
./dataset/images/red Crimson 220 20 60 1.2
./dataset/images/red Crimson 220 20 60 1.3
./dataset/images/red Red 255 0 0 0.9
./dataset/images/red Red 255 0 0 1.0
./dataset/images/red Red 2

./dataset/images/purple DarkOrchid 153 50 204 1.1
./dataset/images/purple DarkOrchid 153 50 204 1.2
./dataset/images/purple DarkOrchid 153 50 204 1.3
./dataset/images/purple DarkMagenta 139 0 139 0.9
./dataset/images/purple DarkMagenta 139 0 139 1.0
./dataset/images/purple DarkMagenta 139 0 139 1.1
./dataset/images/purple DarkMagenta 139 0 139 1.2
./dataset/images/purple DarkMagenta 139 0 139 1.3
./dataset/images/purple Purple 128 0 128 0.9
./dataset/images/purple Purple 128 0 128 1.0
./dataset/images/purple Purple 128 0 128 1.1
./dataset/images/purple Purple 128 0 128 1.2
./dataset/images/purple Purple 128 0 128 1.3
./dataset/images/purple Indigo 75 0 130 0.9
./dataset/images/purple Indigo 75 0 130 1.0
./dataset/images/purple Indigo 75 0 130 1.1
./dataset/images/purple Indigo 75 0 130 1.2
./dataset/images/purple Indigo 75 0 130 1.3
./dataset/images/purple SlateBlue 106 90 205 0.9
./dataset/images/purple SlateBlue 106 90 205 1.0
./dataset/images/purple SlateBlue 106 90 205 1.1
./datase

./dataset/images/blue MediumBlue 0 0 205 0.9
./dataset/images/blue MediumBlue 0 0 205 1.0
./dataset/images/blue MediumBlue 0 0 205 1.1
./dataset/images/blue MediumBlue 0 0 205 1.2
./dataset/images/blue MediumBlue 0 0 205 1.3
./dataset/images/blue DarkBlue 0 0 139 0.9
./dataset/images/blue DarkBlue 0 0 139 1.0
./dataset/images/blue DarkBlue 0 0 139 1.1
./dataset/images/blue DarkBlue 0 0 139 1.2
./dataset/images/blue DarkBlue 0 0 139 1.3
./dataset/images/blue Navy 0 0 128 0.9
./dataset/images/blue Navy 0 0 128 1.0
./dataset/images/blue Navy 0 0 128 1.1
./dataset/images/blue Navy 0 0 128 1.2
./dataset/images/blue Navy 0 0 128 1.3
./dataset/images/blue MidnightBlue 25 25 112 0.9
./dataset/images/blue MidnightBlue 25 25 112 1.0
./dataset/images/blue MidnightBlue 25 25 112 1.1
./dataset/images/blue MidnightBlue 25 25 112 1.2
./dataset/images/blue MidnightBlue 25 25 112 1.3
./dataset/images/brown BurlyWood 222 184 135 0.9
./dataset/images/brown BurlyWood 222 184 135 1.0
./dataset/images/brown

In [15]:
#files_count(images_folder+'yellow')

max_images_count = 1000

for c in next(os.walk(images_folder))[1]:
    full_path = images_folder+c #images_folder+c
    img_count = images_count(full_path) # files_color_count = 
    print(c, img_count)
    
    for i in range(img_count, max_images_count):
        image_to_copy = os.listdir(full_path)[random.randint(0, img_count-1)]
        image_name_to_copy, image_extension = os.path.splitext(image_to_copy)
        
        img_original = Image.open(full_path+'/'+image_to_copy)
        width, height = img_original.size
        
        colors = img_original.getpixel((1, 1)) #((random.randint(0, width-1), random.randint(0, height-1)))
        r = colors[0]
        g = colors[1]
        b = colors[2]
        

        create_image(
            full_path, 
            image_name_to_copy+'_'+str(i), 
            r = r, 
            g = g, 
            b = b, 
            brightness=random.randint(9, 14)/10)        
        
        
        print(c, image_to_copy, image_name_to_copy, image_extension, r, g, b)
        
        if images_count(full_path) >= max_images_count:
            break

'''
    print(c, files_count(images_folder+'yellow')) # 
    

    files_count = files_count(images_folder+c)
    
    for i in range(files_count(images_folder + c), 1200):
        print(c, os.listdir(''.join([images_folder, c]))[1])
'''

blue 200
./dataset/images/blue LightSteelBlue_0_9_200 176 196 222 0.9
blue LightSteelBlue_0_9.png LightSteelBlue_0_9 .png 176 196 222
./dataset/images/blue PowderBlue_1_2_low_201 176 224 230 1.2
blue PowderBlue_1_2_low.png PowderBlue_1_2_low .png 176 224 230
./dataset/images/blue SkyBlue_1_1_202 135 206 235 1.1
blue SkyBlue_1_1.png SkyBlue_1_1 .png 135 206 235
./dataset/images/blue MediumBlue_0_9_low_203 0 0 205 1.4
blue MediumBlue_0_9_low.png MediumBlue_0_9_low .png 0 0 205
./dataset/images/blue DarkBlue_1_2_low_204 0 0 139 1.1
blue DarkBlue_1_2_low.png DarkBlue_1_2_low .png 0 0 139
./dataset/images/blue DodgerBlue_1_3_205 30 144 255 1.0
blue DodgerBlue_1_3.png DodgerBlue_1_3 .png 30 144 255
./dataset/images/blue DarkBlue_1_3_low_206 0 0 139 0.9
blue DarkBlue_1_3_low.png DarkBlue_1_3_low .png 0 0 139
./dataset/images/blue DarkBlue_1_1_low_207 0 0 139 1.3
blue DarkBlue_1_1_low.png DarkBlue_1_1_low .png 0 0 139
./dataset/images/blue DarkBlue_1_3_low_208 0 0 139 0.9
blue DarkBlue_1_3_low

blue DodgerBlue_1_2_low_237_1_4_low_270_1_2.png DodgerBlue_1_2_low_237_1_4_low_270_1_2 .png 30 144 255
./dataset/images/blue DodgerBlue_1_2_low_237_1_4_low_272 30 144 255 1.2
blue DodgerBlue_1_2_low_237_1_4_low.png DodgerBlue_1_2_low_237_1_4_low .png 30 144 255
./dataset/images/blue Cyan_0_9_212_1_1_273 0 255 255 1.2
blue Cyan_0_9_212_1_1.png Cyan_0_9_212_1_1 .png 0 255 255
./dataset/images/blue Cyan_0_9_212_1_1_273_1_2_low_274 0 255 255 1.4
blue Cyan_0_9_212_1_1_273_1_2_low.png Cyan_0_9_212_1_1_273_1_2_low .png 0 255 255
./dataset/images/blue Blue_1_1_263_1_0_low_275 0 0 255 0.9
blue Blue_1_1_263_1_0_low.png Blue_1_1_263_1_0_low .png 0 0 255
./dataset/images/blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276 100 149 237 1.1
blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low.png CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low .png 100 149 237
./dataset/images/blue Aqua_1_3_low_240_1_4_low_277 0 255 255 1.4
blue Aqua_1_3_low_240_1_4_low.png Aqua_1_3_low_240_1_4_low .png 0 255 255
.

blue DarkBlue_1_0_low_319_1_4_low_327_0_9.png DarkBlue_1_0_low_319_1_4_low_327_0_9 .png 0 0 139
./dataset/images/blue CornflowerBlue_1_1_253_0_9_low_314_1_0_low_334 100 149 237 1.2
blue CornflowerBlue_1_1_253_0_9_low_314_1_0_low.png CornflowerBlue_1_1_253_0_9_low_314_1_0_low .png 100 149 237
./dataset/images/blue Aqua_0_9_335 0 255 255 1.4
blue Aqua_0_9.png Aqua_0_9 .png 0 255 255
./dataset/images/blue DarkBlue_1_3_low_304_1_0_low_336 0 0 139 0.9
blue DarkBlue_1_3_low_304_1_0_low.png DarkBlue_1_3_low_304_1_0_low .png 0 0 139
./dataset/images/blue Cyan_0_9_212_1_1_273_1_2_337 0 255 255 1.4
blue Cyan_0_9_212_1_1_273_1_2.png Cyan_0_9_212_1_1_273_1_2 .png 0 255 255
./dataset/images/blue Blue_1_1_263_1_0_low_275_0_9_low_299_1_4_low_338 0 0 255 0.9
blue Blue_1_1_263_1_0_low_275_0_9_low_299_1_4_low.png Blue_1_1_263_1_0_low_275_0_9_low_299_1_4_low .png 0 0 255
./dataset/images/blue DarkTurquoise_1_0_339 0 206 209 1.0
blue DarkTurquoise_1_0.png DarkTurquoise_1_0 .png 0 206 209
./dataset/images/

blue Blue_1_0_230_0_9.png Blue_1_0_230_0_9 .png 0 0 255
./dataset/images/blue Blue_1_2_394 0 0 255 1.4
blue Blue_1_2.png Blue_1_2 .png 0 0 255
./dataset/images/blue Aqua_0_9_222_1_0_395 0 255 255 1.1
blue Aqua_0_9_222_1_0.png Aqua_0_9_222_1_0 .png 0 255 255
./dataset/images/blue Blue_1_1_low_210_1_2_low_317_1_2_low_396 0 0 255 1.0
blue Blue_1_1_low_210_1_2_low_317_1_2_low.png Blue_1_1_low_210_1_2_low_317_1_2_low .png 0 0 255
./dataset/images/blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1_397 100 149 237 1.0
blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1.png CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1 .png 100 149 237
./dataset/images/blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1_low_398 100 149 237 1.0
blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1_low.png CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1_low .png 100 149 237
./dataset/images/blue CornflowerBlue_1_0_low_216_1_1_low_244_0_9_low_276_1_1_397_1_0_low_399 

blue Blue_1_1_263_1_0_low_275_0_9_low_299_1_4_low.png Blue_1_1_263_1_0_low_275_0_9_low_299_1_4_low .png 0 0 255
./dataset/images/blue Blue_1_1_low_210_1_2_low_453 0 0 255 1.2
blue Blue_1_1_low_210_1_2_low.png Blue_1_1_low_210_1_2_low .png 0 0 255
./dataset/images/blue Aqua_1_3_low_240_1_4_low_283_1_4_low_454 0 255 255 1.0
blue Aqua_1_3_low_240_1_4_low_283_1_4_low.png Aqua_1_3_low_240_1_4_low_283_1_4_low .png 0 255 255
./dataset/images/blue Blue_1_0_230_0_9_393_1_2_455 0 0 255 1.4
blue Blue_1_0_230_0_9_393_1_2.png Blue_1_0_230_0_9_393_1_2 .png 0 0 255
./dataset/images/blue Blue_1_1_low_210_1_2_low_317_1_2_456 0 0 255 1.4
blue Blue_1_1_low_210_1_2_low_317_1_2.png Blue_1_1_low_210_1_2_low_317_1_2 .png 0 0 255
./dataset/images/blue Aqua_0_9_222_1_0_low_414_1_3_457 0 255 255 0.9
blue Aqua_0_9_222_1_0_low_414_1_3.png Aqua_0_9_222_1_0_low_414_1_3 .png 0 255 255
./dataset/images/blue Blue_1_1_263_1_0_low_275_0_9_low_299_1_4_low_338_0_9_low_371_1_2_425_1_4_458 0 0 255 1.1
blue Blue_1_1_263_1_0_

...

./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_343_1_1_394_1_3_low_404 255 215 0 1.0
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_343_1_1_394_1_3_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_343_1_1_394_1_3_low .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_373_1_3_405 255 215 0 1.2
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_373_1_3.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_373_1_3 .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_343_1_1_394_1_3_low_404_1_0_low_406 255 215 0 1.1
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_343_1_1_394_1_3_low_404_1_0_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_low_343_1_1_394_1_3_low_404_1_0_low .png 255 2

./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_338_1_1_low_408_1_0_429_1_2_436_1_3_low_438 255 215 0 1.2
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_338_1_1_low_408_1_0_429_1_2_436_1_3_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_low_303_1_2_338_1_1_low_408_1_0_429_1_2_436_1_3_low .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_439 255 215 0 1.0
yellow Gold_0_9_167_1_0_170_0_9.png Gold_0_9_167_1_0_170_0_9 .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low_440 255 215 0 1.3
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_441 255 215 0 0.9
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_

./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_434_1_2_low_456_1_0_low_474 255 215 0 1.3
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_434_1_2_low_456_1_0_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_434_1_2_low_456_1_0_low .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_434_1_2_low_456_1_0_low_475 255 215 0 1.3
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_434_1_2_low_456_1_0_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_434_1_2_low_456_1_0_low .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_447_1_0_476 255 215 0 1.4
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_447_1_0.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_447_1_0 .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_32

yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low_455_1_1_low_472_1_3.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low_455_1_1_low_472_1_3 .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low_455_1_1_low_472_1_3_488_1_0_low_508 255 215 0 1.4
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low_455_1_1_low_472_1_3_488_1_0_low.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_424_1_2_low_455_1_1_low_472_1_3_488_1_0_low .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_509 255 215 0 0.9
yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2.png Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2 .png 255 215 0
./dataset/images/yellow Gold_0_9_167_1_0_170_0_9_220_1_2_290_1_1_low_301_1_2_328_1_2_414_1_2_493_1_0_low_510 255 2

"\n    print(c, files_count(images_folder+'yellow')) # \n    \n\n    files_count = files_count(images_folder+c)\n    \n    for i in range(files_count(images_folder + c), 1200):\n        print(c, os.listdir(''.join([images_folder, c]))[1])\n"